In [52]:
import dataRead
import dataClean
import runTrain
import runTest
import writeResult

In [53]:
# Read data
reload(dataRead)
X_train,X_test = dataRead.do()

In [8]:
# Test Above function here


In [54]:
reload(dataClean)
X_train_features,X_test_features = dataClean.do(X_train,X_test)

In [ ]:
# Test Above function here

In [55]:
# Train model
reload(runTrain)
runTrain.do(X_train_features)

In [6]:
# Test Above function here


In [68]:
# Test model
reload(runTest)
result = runTest.do(X_test_features)

In [69]:
# Test Above function here


In [70]:
# Write Results
reload(writeResult)
writeResult.do(result,X_test_features)

In [ ]:
# Test Above function here